# Moroccan plates detection 

# Participants:
Abderrahman Skiredj & Ferdaous Azhari

# Project goal:
Automatic identification of the license plate string of the car(s) in a given image

## Pipeline :
### Step 0 (optional, used only if step 1 didnt succeed): Vehicle detection
Read the image then detect the sub-images corresponding to vehicles
### Step 1: Plate detection
Read the image then detect its sub-images corresponding to the plates
### Step 2: Characters detection
For each subimage correspondig to a plate, segment it in its turn to subimages corresponding to characters
### Step 3: Pate string recognition
For each sub-image corresponding to a character, recognize the character written in it

# Existing open source projects
we have leveraged the following existing projects https://github.com/HamzaEzzRa/MLPDR (for step 1 , 2 and 3), the project https://techvidvan.com/tutorials/opencv-vehicle-detection-classification-counting/ (for step 0)

# Limitations of leveraged projects and proposed solutions
- the project https://github.com/HamzaEzzRa/MLPDR cant detect plates of cars that are far away in the image. In this case we enhance it with the aforementioned step 0
- It didnt succeed detecting plates with black background (they are those plates with the letter "m" or "almaghrib" typically for dacia duster car). The solution is to detect such a case, deduce that the letter character is red (it's always the case) and is either "m" or "jim",  invert the image (take its negative) to retrieve the digits
- For plates that are not well contrasted/brighted, the characters couldnt be identified. The proposed solution is to loop on many possible brightness values and contrast values then modify the image according to these values then predict the plates and finally take the prediction with highest confidence
- Finally the project couldnt predict correctly when plates are of squared forms (with 2 lines of characters). Our idea was to detect such cases (it is easy as the detected characters have overlapping x axis), then take the characters of the second line first then the characters of the first line. We couldnt implement the idea in the allotted time

# The code

In [ ]:
#Install requirements
!pip install -r requirements.txt

In [1]:
requirements_str='''
arabic-reshaper==2.1.3
cycler==0.10.0
decorator==4.4.2
future==0.18.2
imageio==2.9.0
imutils==0.5.4
kiwisolver==1.3.1
matplotlib==3.4.1
networkx==2.5.1
numpy==1.20.2
opencv-contrib-python==4.5.1.48
opencv-python==4.5.2.52
Pillow==8.2.0
pyparsing==2.4.7
PyQt5==5.15.4
PyQt5-Qt5==5.15.2
PyQt5-sip==12.9.0
pytesseract==0.3.7
python-bidi==0.4.2
python-dateutil==2.8.1
PyWavelets==1.1.1
scikit-image==0.18.1
scipy==1.6.3
six==1.15.0
tifffile==2021.4.8'''

In [7]:
print(requirements_str.replace('\n','\n!pip install '))


!pip install arabic-reshaper==2.1.3
!pip install cycler==0.10.0
!pip install decorator==4.4.2
!pip install future==0.18.2
!pip install imageio==2.9.0
!pip install imutils==0.5.4
!pip install kiwisolver==1.3.1
!pip install matplotlib==3.4.1
!pip install networkx==2.5.1
!pip install numpy==1.20.2
!pip install opencv-contrib-python==4.5.1.48
!pip install opencv-python==4.5.2.52
!pip install Pillow==8.2.0
!pip install pyparsing==2.4.7
!pip install PyQt5==5.15.4
!pip install PyQt5-Qt5==5.15.2
!pip install PyQt5-sip==12.9.0
!pip install pytesseract==0.3.7
!pip install python-bidi==0.4.2
!pip install python-dateutil==2.8.1
!pip install PyWavelets==1.1.1
!pip install scikit-image==0.18.1
!pip install scipy==1.6.3
!pip install six==1.15.0
!pip install tifffile==2021.4.8


# Helper functions for step 0

In [ ]:
import cv2
import numpy as np
from tracker import *

# Initialize Tracker
tracker = EuclideanDistTracker()

input_size = 320

# Detection confidence threshold
confThreshold = 0.2
nmsThreshold = 0.2

font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 2

# Middle cross line position
middle_line_position = 225
up_line_position = middle_line_position - 15
down_line_position = middle_line_position + 15

# Store Coco Names in a list
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')

# class index for our required detection classes
required_class_index = [2,3,7]

detected_classNames = []

## Model Files
modelConfiguration = 'yolov3-320.cfg'
modelWeigheights = 'yolov3-320.weights'

# configure the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

# Configure the network backend

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')


# Function for finding the center of a rectangle
def find_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy


# List for store vehicle count information
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]


# Function for finding the detected objects from the network output
def postProcess(outputs, img, img_path):
    global detected_classNames
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if classId in required_class_index:
                if confidence > confThreshold:
                    # print(classId)
                    w, h = int(det[2] * width), int(det[3] * height)
                    x, y = int((det[0] * width) - w / 2), int((det[1] * height) - h / 2)
                    boxes.append([x, y, w, h])
                    classIds.append(classId)
                    confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
    # print(classIds)
    if np.size(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]

            name = classNames[classIds[i]]
            detected_classNames.append(name)
            detection.append([x, y, w, h, required_class_index.index(classIds[i])])


    images = [img[z[1]: z[1]+z[3], z[0]: z[0]+z[2]] for z in detection]
    return images



def from_static_image(img_path):
    img = cv2.imread(img_path)

    blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

    # Set the input of the network
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
    # Feed data to the network
    outputs = net.forward(outputNames)

    # Find the objects from the network output
    return postProcess(outputs, img, img_path)